In [ ]:
%reset -f
import numpy as np
from landlab import RasterModelGrid
from landlab.components import OverlandFlow, FlowAccumulator, SpatialPrecipitationDistribution
from landlab.plot.imshow import imshow_grid
import matplotlib.pyplot as plt


In [ ]:
#Define grid
# here we use an arbitrary, very small, "real" catchment
fname = 'hugo_site.asc'
rmg, z = read_esri_ascii(fname, name='topographic__elevation')
rmg.status_at_node[mg.nodes_at_right_edge] = mg.BC_NODE_IS_FIXED_VALUE
rmg.status_at_node[np.isclose(z, -9999.)] = mg.BC_NODE_IS_CLOSED

plt.figure()
imshow_grid_at_node(mg, z, colorbar_label='Elevation (m)')
plt.show()